In [49]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Minas Gerais - Produção de Cimento (t)', axis=1)
data

,Ano,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Consumo de Cimento (t)
0,2008,0.738122,0.717622,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.765604,5.483335e+08,...,11.760983,0.846543,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497,5902.354000
1,2009,0.737078,0.721167,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.767083,5.302764e+08,...,9.472585,0.848819,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866,6032.354000
2,2010,0.736297,0.724635,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.768464,5.046425e+08,...,9.332485,0.850522,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811,6751.429000
3,2011,0.735777,0.728021,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.769750,4.854622e+08,...,10.985689,0.851665,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000,7134.988000
4,2012,0.735118,0.730837,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770583,4.867661e+08,...,8.159014,0.852207,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,7144.026000
5,2013,0.735878,0.734823,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.772298,5.225848e+08,...,7.913561,0.852224,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054,7225.971000
6,2014,0.736117,0.738392,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.773490,5.676413e+08,...,10.350442,0.851826,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659,6631.735714
7,2015,0.735877,0.740091,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.773781,5.179392e+08,...,12.454398,0.850979,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753,5836.891371
8,2016,0.730090,0.749468,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778559,4.524144e+08,...,13.111908,0.862294,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880,5091.098914
9,2017,0.730553,0.750419,3.480263e+08,1.759670e+07,14.339149,2.842415e+08,9.789036,0.778792,4.213689e+08,...,9.479812,0.861503,20.552463,889.087988,1271.437296,18.605289,13.974131,76.845115,1.808195,4898.010000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.738122,0.717622,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.765604,5.483335e+08,2.546644e+09,...,2.868108,11.760983,0.846543,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497
1,0.737078,0.721167,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.767083,5.302764e+08,2.679945e+09,...,2.305885,9.472585,0.848819,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866
2,0.736297,0.724635,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.768464,5.046425e+08,2.891930e+09,...,1.958246,9.332485,0.850522,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811
3,0.735777,0.728021,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.769750,4.854622e+08,3.091988e+09,...,2.079088,10.985689,0.851665,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000
4,0.735118,0.730837,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770583,4.867661e+08,3.189506e+09,...,2.183282,8.159014,0.852207,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120
5,0.735878,0.734823,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.772298,5.225848e+08,3.093872e+09,...,2.403445,7.913561,0.852224,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054
6,0.736117,0.738392,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.773490,5.676413e+08,2.779340e+09,...,2.748397,10.350442,0.851826,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659
7,0.735877,0.740091,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.773781,5.179392e+08,2.561731e+09,...,2.501529,12.454398,0.850979,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753
8,0.730090,0.749468,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778559,4.524144e+08,2.516859e+09,...,2.073464,13.111908,0.862294,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880
9,0.730553,0.750419,3.480263e+08,1.759670e+07,14.339149,2.842415e+08,9.789036,0.778792,4.213689e+08,2.497710e+09,...,2.177386,9.479812,0.861503,20.552463,889.087988,1271.437296,18.605289,13.974131,76.845115,1.808195


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7225.971000
5     6631.735714
6     5836.891371
7     5091.098914
8     4898.010000
9     5509.999000
10    6256.450000
11    6954.530000
12    6975.675000
13    6706.893000
14            NaN
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.323232,-1.620866,-1.838149,0.673524,-0.408054,-2.145004,-1.321648,-1.614153,1.204752,-0.990234,...,2.026352,0.984445,-1.468389,0.546435,-1.229262,-1.471435,-0.707157,0.340929,0.384164,0.760640
1,0.960748,-1.314156,-1.689631,1.223073,-0.189336,-2.066959,-1.288472,-1.285800,0.820657,-0.546688,...,0.085558,-0.051710,-1.017356,0.681014,-1.970482,-1.167751,-0.184239,0.652313,0.732526,0.855996
2,0.689922,-1.014059,-1.069034,1.286693,1.395717,-0.274561,-1.329227,-0.979439,0.275392,0.158672,...,-1.114490,-0.115145,-0.679745,0.748566,-1.479584,-0.829204,-1.733683,0.662956,1.008690,1.063440
3,0.509488,-0.721005,-0.540878,1.182986,1.874830,0.623361,-1.340601,-0.693916,-0.132596,0.824346,...,-0.697344,0.633403,-0.453278,-0.081023,-0.245950,-0.944407,-0.009940,0.941327,1.189931,1.063951
4,0.280737,-0.477317,-0.095331,0.897610,1.388579,0.961592,-0.117537,-0.509072,-0.104860,1.148829,...,-0.337667,-0.646476,-0.345909,-1.282156,0.057146,-0.449809,0.790783,0.731347,0.794405,0.743448
5,0.544382,-0.132344,0.271919,0.386180,0.533962,0.959671,0.399829,-0.128412,0.657046,0.830615,...,0.422337,-0.757614,-0.342490,-2.222830,0.603696,0.686946,-0.484821,0.064056,0.058673,0.338865
6,0.627589,0.176473,0.539068,-0.237305,-0.405597,0.515649,0.911621,0.136019,1.615453,-0.215960,...,1.613108,0.345772,-0.421353,-0.696635,0.979423,1.228615,1.884804,-0.001086,-0.360323,0.052022
7,0.544238,0.323529,0.732633,-0.871091,-1.189838,0.081036,1.299648,0.200563,0.558230,-0.940033,...,0.760923,1.298413,-0.589358,0.119948,1.063393,1.342416,1.710518,0.013713,-0.355502,-0.096007
8,-1.463830,1.134917,0.899544,-1.196495,-1.175976,0.155328,1.209478,1.261018,-0.835563,-1.089341,...,-0.716758,1.596124,1.653000,0.150363,0.918566,1.263379,-0.892435,-0.182621,-0.273294,-0.219681
9,-1.303270,1.217266,0.989637,-1.290727,-0.710864,0.464412,0.692295,1.312653,-1.495939,-1.153056,...,-0.358019,-0.048438,1.496166,1.537521,0.818685,0.932975,-0.448342,-0.021892,-0.122112,-0.847007


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7225.971000
5     6631.735714
6     5836.891371
7     5091.098914
8     4898.010000
9     5509.999000
10    6256.450000
11    6954.530000
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.340422,0.984852,0.49047,-0.775222,-1.103617,-0.390774,0.251054,0.738141,1.108405,2.635823,...,-1.482779,-2.260447,0.453072,-0.236248,-1.371702,-1.653764,-1.48158,-3.104735,-2.983429,-1.314068


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    6975.675
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [45]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    spli_cut_size=len(train_input)//6 + 1
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-spli_cut_size:], train_target.iloc[-spli_cut_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [46]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [47]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[780724331, 1155999820, 2401474664, 1879622455, 1711878552, 2467150894, 1122410835, 447807543, 4233694502, 3864571529, 4051353717, 3651156139, 1910255618, 1001925453, 4024844228, 4249993407, 1155868569, 746771986, 2972679603, 651130362, 3082942601, 205369448, 3887049568, 853213067, 2459506290, 170145183, 4192387047, 550966827, 382553341, 1874599887, 2468144090, 2357789238, 3992760051, 3087056008, 1068406864, 2564796536, 2448206262, 77979810, 2875943504, 3703308918, 2577801594, 1952429651, 2468978736, 1846198667, 84865676, 3612537891, 331595392, 3932813578, 1371534499, 2008900355, 2874607574, 3523123813, 478418901, 2245647374, 1011776854, 1261010095, 1436137118, 4202280570, 3435302927, 1670637750, 803895258, 1234677404, 3188638678, 2408462825, 1830169002, 4039900396, 4104277013, 3705633948, 176139661, 4221779297, 1430653708, 3711886716, 1001412309, 1777646988, 731191142, 2606164390, 3993860570, 3664442256, 1694913995, 3274696597, 1921354941, 3836946211, 1580519715, 2962032125, 363666486

val_loss: 457668.84375


Step: 89 ___________________________________________
val_loss: 467103.75


Step: 90 ___________________________________________
val_loss: 395118.34375


Step: 91 ___________________________________________
val_loss: 437981.03125


Step: 92 ___________________________________________
val_loss: 453850.09375


Step: 93 ___________________________________________
val_loss: 441566.65625


Step: 94 ___________________________________________
val_loss: 321105.125


Step: 95 ___________________________________________
val_loss: 418620.375


Step: 96 ___________________________________________
val_loss: 477235.28125


Step: 97 ___________________________________________
val_loss: 564671.8125


Step: 98 ___________________________________________
val_loss: 377097.34375


Step: 99 ___________________________________________
val_loss: 543191.1875


final_seed: 1234677404


In [50]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 507ms/step - loss: 40164796.0000 - val_loss: 38786192.0000
Epoch 2/10000
1/1 [==============================] - 0s 16ms/step - loss: 39499340.0000 - val_loss: 40101920.0000
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 38267300.0000 - val_loss: 46270052.0000
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 36057340.0000 - val_loss: 39830988.0000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 36908016.0000 - val_loss: 45013532.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 33624876.0000 - val_loss: 56573024.0000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 34264356.0000 - val_loss: 42197108.0000
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 34182264.0000 - val_loss: 38635092.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 33505974.0000

1/1 [==============================] - 0s 15ms/step - loss: 8893109.0000 - val_loss: 9647341.0000
Epoch 73/10000
1/1 [==============================] - 0s 15ms/step - loss: 7918178.5000 - val_loss: 8288892.5000
Epoch 74/10000
1/1 [==============================] - 0s 15ms/step - loss: 9365028.0000 - val_loss: 7340214.5000
Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 10743741.0000 - val_loss: 7261306.0000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 7622988.5000 - val_loss: 9429698.0000
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 8227305.5000 - val_loss: 7101604.0000
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 8837457.0000 - val_loss: 6871876.5000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 6964049.5000 - val_loss: 7166538.5000
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 8494290.0000 - val_loss: 6214486.50

1/1 [==============================] - 0s 14ms/step - loss: 2291432.5000 - val_loss: 2557208.5000
Epoch 145/10000
1/1 [==============================] - 0s 14ms/step - loss: 2510058.0000 - val_loss: 2829157.7500
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 2567067.5000 - val_loss: 2923195.2500
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 1948995.8750 - val_loss: 3282151.2500
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 2939636.7500 - val_loss: 2473915.0000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 1944576.3750 - val_loss: 3431705.0000
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 2709649.2500 - val_loss: 2462335.2500
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 2233053.0000 - val_loss: 2616579.0000
Epoch 152/10000
1/1 [==============================] - 0s 15ms/step - loss: 1989903.1250 - val_loss: 262

1/1 [==============================] - 0s 14ms/step - loss: 1023058.1875 - val_loss: 1099702.6250
Epoch 217/10000
1/1 [==============================] - 0s 14ms/step - loss: 546394.3125 - val_loss: 835470.1875
Epoch 218/10000
1/1 [==============================] - 0s 15ms/step - loss: 504895.8438 - val_loss: 830503.5000
Epoch 219/10000
1/1 [==============================] - 0s 15ms/step - loss: 879043.8125 - val_loss: 877473.4375
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 1082786.7500 - val_loss: 506367.5938
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 1078013.6250 - val_loss: 507700.6562
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 1235320.3750 - val_loss: 874258.8125
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 984418.2500 - val_loss: 437186.0000
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 1177563.1250 - val_loss: 272249.0938
Ep

1/1 [==============================] - 0s 14ms/step - loss: 601603.1250 - val_loss: 468733.0938
Epoch 290/10000
1/1 [==============================] - 0s 14ms/step - loss: 586010.9375 - val_loss: 477106.8750
Epoch 291/10000
1/1 [==============================] - 0s 15ms/step - loss: 460857.1562 - val_loss: 448258.0000
Epoch 292/10000
1/1 [==============================] - 0s 15ms/step - loss: 333582.9062 - val_loss: 503492.4688
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 425621.2812 - val_loss: 495180.8438
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 320961.2188 - val_loss: 481753.7188
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 355684.7500 - val_loss: 425830.3438
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 595368.1875 - val_loss: 647374.1875
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 471521.7188 - val_loss: 449150.1562
Epoch 29

...

1/1 [==============================] - 0s 14ms/step - loss: 209954.6719 - val_loss: 135573.0781
Epoch 947/10000
1/1 [==============================] - 0s 15ms/step - loss: 298839.4062 - val_loss: 154431.0156
Epoch 948/10000
1/1 [==============================] - 0s 15ms/step - loss: 170115.8906 - val_loss: 146015.5156
Epoch 949/10000
1/1 [==============================] - 0s 14ms/step - loss: 282477.7500 - val_loss: 105552.7188
Epoch 950/10000
1/1 [==============================] - 0s 15ms/step - loss: 283816.3750 - val_loss: 138661.8906
Epoch 951/10000
1/1 [==============================] - 0s 14ms/step - loss: 247551.8125 - val_loss: 153364.7344
Epoch 952/10000
1/1 [==============================] - 0s 15ms/step - loss: 192173.3906 - val_loss: 116891.0391
Epoch 953/10000
1/1 [==============================] - 0s 15ms/step - loss: 241538.5781 - val_loss: 118020.1328
Epoch 954/10000
1/1 [==============================] - 0s 14ms/step - loss: 251170.7344 - val_loss: 149585.8906
Epoch 95

1/1 [==============================] - 0s 14ms/step - loss: 164285.7969 - val_loss: 152151.0469
Epoch 1020/10000
1/1 [==============================] - 0s 15ms/step - loss: 249865.2969 - val_loss: 116486.4609
Epoch 1021/10000
1/1 [==============================] - 0s 14ms/step - loss: 399113.7812 - val_loss: 127113.0938
Epoch 1022/10000
1/1 [==============================] - 0s 15ms/step - loss: 222521.0781 - val_loss: 169507.3906
Epoch 1023/10000
1/1 [==============================] - 0s 15ms/step - loss: 189182.2969 - val_loss: 125849.6172
Epoch 1024/10000
1/1 [==============================] - 0s 14ms/step - loss: 194548.0781 - val_loss: 122696.6875
Epoch 1025/10000
1/1 [==============================] - 0s 15ms/step - loss: 308264.1562 - val_loss: 131541.8125
Epoch 1026/10000
1/1 [==============================] - 0s 15ms/step - loss: 182492.1875 - val_loss: 132815.0938
Epoch 1027/10000
1/1 [==============================] - 0s 15ms/step - loss: 259236.7969 - val_loss: 130033.9453


1/1 [==============================] - 0s 15ms/step - loss: 247234.3750 - val_loss: 210940.6406
Epoch 1092/10000
1/1 [==============================] - 0s 16ms/step - loss: 214206.3281 - val_loss: 185149.1406
Epoch 1093/10000
1/1 [==============================] - 0s 15ms/step - loss: 202506.7969 - val_loss: 157003.3750
Epoch 1094/10000
1/1 [==============================] - 0s 15ms/step - loss: 219291.6094 - val_loss: 167448.6562
Epoch 1095/10000
1/1 [==============================] - 0s 14ms/step - loss: 313163.2188 - val_loss: 160214.8125
Epoch 1096/10000
1/1 [==============================] - 0s 15ms/step - loss: 314341.0938 - val_loss: 157370.0469
Epoch 1097/10000
1/1 [==============================] - 0s 15ms/step - loss: 201356.6250 - val_loss: 175115.6094
Epoch 1098/10000
1/1 [==============================] - 0s 15ms/step - loss: 244050.5469 - val_loss: 202668.9844
Epoch 1099/10000
1/1 [==============================] - 0s 15ms/step - loss: 232598.3750 - val_loss: 164839.9219


1/1 [==============================] - 0s 15ms/step - loss: 276519.2812 - val_loss: 134030.6094
Epoch 1164/10000
1/1 [==============================] - 0s 15ms/step - loss: 185150.5625 - val_loss: 130686.3203
Epoch 1165/10000
1/1 [==============================] - 0s 15ms/step - loss: 251152.9219 - val_loss: 157481.1406
Epoch 1166/10000
1/1 [==============================] - 0s 15ms/step - loss: 245055.0625 - val_loss: 142542.9375
Epoch 1167/10000
1/1 [==============================] - 0s 14ms/step - loss: 223950.3125 - val_loss: 140035.4688
Epoch 1168/10000
1/1 [==============================] - 0s 14ms/step - loss: 197127.2656 - val_loss: 137829.8594
Epoch 1169/10000
1/1 [==============================] - 0s 14ms/step - loss: 254248.8906 - val_loss: 168782.0469
Epoch 1170/10000
1/1 [==============================] - 0s 14ms/step - loss: 208422.6719 - val_loss: 151787.5000
Epoch 1171/10000
1/1 [==============================] - 0s 14ms/step - loss: 149914.8906 - val_loss: 148043.9844


1/1 [==============================] - 0s 14ms/step - loss: 217167.9219 - val_loss: 170957.6719
Epoch 1236/10000
1/1 [==============================] - 0s 14ms/step - loss: 211483.0000 - val_loss: 162707.9531
Epoch 1237/10000
1/1 [==============================] - 0s 15ms/step - loss: 260912.3281 - val_loss: 159908.7500
Epoch 1238/10000
1/1 [==============================] - 0s 14ms/step - loss: 227368.7969 - val_loss: 164017.6562
Epoch 1239/10000
1/1 [==============================] - 0s 14ms/step - loss: 197813.3750 - val_loss: 158347.3594
Epoch 1240/10000
1/1 [==============================] - 0s 15ms/step - loss: 239454.7344 - val_loss: 193671.2344
Epoch 1241/10000
1/1 [==============================] - 0s 15ms/step - loss: 260073.5156 - val_loss: 138188.9219
Epoch 1242/10000
1/1 [==============================] - 0s 15ms/step - loss: 206588.7656 - val_loss: 146825.5312
Epoch 1243/10000
1/1 [==============================] - 0s 15ms/step - loss: 253013.7500 - val_loss: 185585.8281


In [51]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 44ms/step
[6975.675](test_target) - [[6521.004]](prediction) = 454.6710937500002


In [52]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [53]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.279372,-1.229169,-0.918215,-1.407905,-0.838794,-0.751794,-0.482076,-1.238643,1.148205,-1.122043,...,1.308949,1.412346,-1.279057,-1.335859,1.073554,-1.201994,0.260829,-1.413615,-1.269117,-1.049915
1,-0.117760,0.008898,-0.472376,0.588399,-0.566665,-0.661457,1.392429,0.028285,0.140893,-0.184474,...,-0.190804,-0.643250,0.117025,0.265921,-1.334029,-0.044300,1.073320,0.671180,0.094182,-0.295566
2,-1.161611,1.220272,1.390591,0.819506,1.405459,1.413251,-0.910354,1.210357,-1.289098,1.306518,...,-1.118144,-0.769096,1.162032,1.069938,0.260475,1.246294,-1.334149,0.742435,1.174935,1.345481


train_target:


,Minas Gerais - Consumo de Cimento (t)
0,6032.354
1,6751.429
2,7134.988


test_input:


...

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.340422,0.984852,0.49047,-0.775222,-1.103617,-0.390774,0.251054,0.738141,1.108405,2.635823,...,-1.482779,-2.260447,0.453072,-0.236248,-1.371702,-1.653764,-1.48158,-3.104735,-2.983429,-1.314068


test_target:


,Minas Gerais - Consumo de Cimento (t)
12,6975.675


1/1 [==============================] - 0s 33ms/step
Error: 454.6710937500002




[7135.0234375,
 7144.013671875,
 7200.35107421875,
 6671.0224609375,
 5839.712890625,
 5484.3349609375,
 4987.5615234375,
 5245.689453125,
 5540.64794921875,
 6521.00390625]

In [54]:
display(targets)
display(predictions)

[7144.026,
 7225.971,
 6631.735714285716,
 5836.891371428574,
 5091.098914285716,
 4898.01,
 5509.999,
 6256.45,
 6954.53,
 6975.675]

[7135.0234375,
 7144.013671875,
 7200.35107421875,
 6671.0224609375,
 5839.712890625,
 5484.3349609375,
 4987.5615234375,
 5245.689453125,
 5540.64794921875,
 6521.00390625]

In [55]:
mae = mean_absolute_error(predictions, targets)
mae

623.0396445312492

In [56]:
porcentage = mae/np.mean(targets)
porcentage

0.099647461482709